# Tutorial 01: Validating models

In this notebook, we can see how to validate models. Note that it requires to have run the TUTORIAL_00 notebook first so we precompute the features that will be used in this notebook.

In [1]:
import os
import random
import argparse
import json
from time import time
import pandas as pd
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.errors import ResourceExhaustedError

from damage.data import DataStream
from damage.models import CNN, RandomSearch, CNNPreTrained

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
RESULTS_PATH = '../logs/experiments'
FEATURES_PATH = '../logs/features'
features_file_name = 'test_daraa.p'

In [3]:
features = pd.read_pickle('{}/{}'.format(FEATURES_PATH, features_file_name)).dropna(subset=['destroyed'])
features.head()

annotation_date  damage_num  destroyed   latitude  \
city  patch_id   date                                                           
daraa 10080-8224 2017-02-07      2016-04-19         2.0        0.0  32.616861   
      10208-8288 2017-02-07      2016-04-19         1.0        0.0  32.616517   
      10272-4768 2017-02-07      2016-04-19         1.0        0.0  32.635400   
      10336-7904 2017-02-07      2016-04-19         3.0        1.0  32.618577   
      10400-8160 2017-02-07      2016-04-19         2.0        0.0  32.617204   

                             longitude  \
city  patch_id   date                    
daraa 10080-8224 2017-02-07  36.122191   
      10208-8288 2017-02-07  36.122878   
      10272-4768 2017-02-07  36.123221   
      10336-7904 2017-02-07  36.123565   
      10400-8160 2017-02-07  36.123908   

                                                                         image  
city  patch_id   date                                                           
daraa 10080-8224 2017-02-07  [[[99, 85, 74, 90, 65, 58], [99, 81, 74, 90, 6...  
      10208-8288 2017-02-07  [[[123, 125, 123, 90, 61, 58], [107, 113, 115,...  
      10272-4768 2017-02-07  [[[99, 73, 66, 41, 40, 41], [99, 73, 66, 41, 4...  
      10336-7904 2017-02-07  [[[181, 190, 197, 132, 89, 74], [181, 186, 189...  
      10400-8160 2017-02-07  [[[99, 97, 99, 49, 45, 41], [82, 85, 82, 49, 4...

In [4]:
len(features)

830

We will make use of three custom classes: __RandomSearch__, __CNN__ and __DataStream__. __RandomSearch__ is a class that samples hyperparameters for ML models. As of may 2019, only the space for cnn's has been implemented. __CNN__ is a class that defines a Convolutional Neural Network model and follows the standards of Sklearn and Keras APIs, containing methods called fit, predict, fit_generator, predict_generator and validate_generator. In this case, we make use of the validate_generator method, which takes a generator of data as required by Keras's fit_generator method: each batch yields a tuple of (features, target). We use the __DataStream__ object to create these generators, first by generating the indices with the split_by_path_id method, which follows the standards of Sklearn splitters and then with the get_data_generator_from_index method that turns those indices into data generators.

In [7]:
# Modelling
sampler = RandomSearch()
models = {
    CNN:sampler.sample_cnn,
    CNNPreTrained: sampler.sample_cnn_pretrained,
}
Model = random.choice([CNN])
sample_func = models[Model]
spaces = sample_func(1)
# # Do splits
# class_proportion = {
#     1: 0.3,
# }
batch_size = spaces[0]['batch_size']
test_batch_size = batch_size
train_proportion = 0.7
data_stream = DataStream(batch_size = batch_size, train_proportion=train_proportion, test_batch_size=test_batch_size)
unique_patches = features.index.get_level_values('patch_id').unique().tolist()
train_patches = random.sample(unique_patches, round(len(unique_patches)*train_proportion))
train_data = features.loc[features.index.get_level_values('patch_id').isin(train_patches)]
# if train_data['destroyed'].mean() > class_proportion[1]:
#     train_data_upsampled = train_data.copy()
# else:
#     train_data_upsampled = data_stream._upsample_class_proportion(train_data, class_proportion).sample(frac=1)
test_patches = list(set(unique_patches) - set(train_patches))
test_data = features.loc[features.index.get_level_values('patch_id').isin(test_patches)]

train_indices = data_stream._get_index_generator(train_data, batch_size)
test_indices = data_stream._get_index_generator(test_data, test_batch_size)
train_generator = data_stream.get_train_data_generator_from_index(
    [train_data['image'], train_data['destroyed']], train_indices)
test_generator = data_stream.get_train_data_generator_from_index(
    [test_data['image'], test_data['destroyed']], test_indices)
train_dataset = Dataset.from_generator(lambda: train_generator, (tf.float32, tf.int32))
test_dataset = Dataset.from_generator(lambda: test_generator, (tf.float32, tf.int32))
num_batches = len(train_indices)
num_batches_test = len(test_indices)
print(spaces[0])

W0723 18:35:32.917455 4541539776 deprecation.py:323] From /Users/jordi/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


{'dense_units': 128, 'batch_size': 33, 'convolutional_layers': [{'kernel_size': [7, 7], 'pool_size': [6, 6], 'filters': 32, 'dropout': 0.33333333333333337, 'activation': 'relu'}, {'kernel_size': [7, 7], 'pool_size': [6, 6], 'filters': 64, 'dropout': 0.33333333333333337, 'activation': 'relu'}, {'kernel_size': [7, 7], 'pool_size': [6, 6], 'filters': 128, 'dropout': 0.33333333333333337, 'activation': 'relu'}], 'epochs': 6, 'layer_type': 'cnn', 'class_weight': 1.15, 'learning_rate': 0.0017575106248547913}


In [ ]:
#Validate
for space in spaces:
    print('Now validating:\n')
    print(space)
    try:
        model = Model(**space)
        losses = model.validate_generator(train_dataset, test_dataset,
                                          steps_per_epoch=num_batches,
                                          validation_steps=num_batches_test,
                                          **space)
    except Exception as e:
        losses = {'log': str(e)}

    losses['model'] = str(Model)
    losses['space'] = space
    losses['features'] = features_file_name
    losses['num_batches_train'] = num_batches
    losses['num_batches_test'] = num_batches_test
    identifier = round(time())
   

In [15]:
with open('{}/experiment_{}.json'.format(RESULTS_PATH, identifier), 'w') as f:
        json.dump(str(losses), f)
if 'val_recall_positives' in losses.keys():
    if losses['val_recall_positives'][-1] > 0.4 and losses['val_precision_positives'][-1] > 0.1:
        model.save('../logs/models/model_{}.h5'.format(identifier))